In [9]:
import pandas as pd
import numpy as np
import os
import datetime

* This dataset is already pre-processed by Iowa State University
* Simply create path names for all METAR files and concatenate the dataframes together

In [10]:
metar_paths = ['METAR_Final/' + file for file in os.listdir('METAR_Final')]

In [11]:
metar_dfs = []
for path in metar_paths:
    df = pd.read_csv(path)
    metar_dfs.append(df)

In [12]:
metar_df = pd.concat(metar_dfs)

* Converting the datetime objects and truncating minutes to have METAR conditions span hour-long period

In [13]:
metar_df['valid'] = pd.to_datetime(metar_df['valid']).dt.floor('H')
metar_df = metar_df.drop_duplicates(subset = 'valid', keep='first').reset_index()

In [14]:
metar_df['valid'].value_counts()

2022-01-01 00:00:00    1
2022-08-31 18:00:00    1
2022-08-31 12:00:00    1
2022-08-31 13:00:00    1
2022-08-31 14:00:00    1
                      ..
2022-05-02 02:00:00    1
2022-05-02 01:00:00    1
2022-05-02 00:00:00    1
2022-05-01 23:00:00    1
2022-12-30 23:00:00    1
Name: valid, Length: 8735, dtype: int64

In [15]:
metar_df.head()

,index,station,valid,lon,lat,elevation,tmpf,dwpf,relh,drct,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,0,LAX,2022-01-01 00:00:00,-118.3865,33.9382,32.0,57.0,48.90,74.27,280.00,...,M,M,M,M,M,M,M,57.00,KLAX 010053Z 28012KT 10SM FEW015 SCT038 14/09 ...,M
1,1,LAX,2022-01-01 01:00:00,-118.3865,33.9382,32.0,55.9,48.90,77.28,M,...,M,M,M,M,M,M,M,55.90,KLAX 010153Z VRB04KT 10SM FEW012 SCT034 13/09 ...,M
2,2,LAX,2022-01-01 02:00:00,-118.3865,33.9382,32.0,55.0,48.00,77.21,350.00,...,M,M,M,M,M,M,M,55.00,KLAX 010253Z 35006KT 10SM SCT014 SCT049 13/09 ...,M
3,3,LAX,2022-01-01 03:00:00,-118.3865,33.9382,32.0,54.0,41.00,61.32,360.00,...,M,M,M,M,M,M,M,54.00,KLAX 010353Z 36004KT 10SM FEW013 SCT044 12/05 ...,M
4,4,LAX,2022-01-01 04:00:00,-118.3865,33.9382,32.0,54.0,37.00,52.45,M,...,M,M,M,M,M,M,M,54.00,KLAX 010453Z VRB05KT 10SM SCT013 BKN030 12/03 ...,M


# Reading in IFF data from chunked IFF folder

In [16]:
chunked_iff_paths = ['Sherlock_data/chunked_IFF/' + file for file in os.listdir('Sherlock_data/chunked_IFF') if file.endswith('csv')]
chunked_iffs = []
for path in chunked_iff_paths:
    df = pd.read_csv(path)
    sig_5 = df['significance']
    chunked_iffs.append(df)

C:\Users\abrun\AppData\Local\Temp\ipykernel_7792\2032884243.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\abrun\AppData\Local\Temp\ipykernel_7792\2032884243.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\abrun\AppData\Local\Temp\ipykernel_7792\2032884243.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\abrun\AppData\Local\Temp\ipykernel_7792\2032884243.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\abrun\AppData\Local\Temp\ipykernel_7792\2032884243.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
C:\Users\abrun\AppData\Local\Temp\ipykernel_7

In [17]:
iff_df = pd.concat(chunked_iffs)
# Renaming to seconds since midnight 1/1/70
iff_df = iff_df.drop('Unnamed: 0', axis=1)

In [18]:
iff_df['mergeKey'].isnull().any()

False

In [19]:
iff_df.head(1)

,fltKey,AcId,acType,Orig,Dest,perfCat,pKey_fltAcMode,coord1,coord2,alt,...,coord1Accur,coord2Accur,groundSpeed,course,rateOfClimb,scratchPad,airportCode,trackNumber,recTime,mergeKey
0,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93549,-118.5447,21.69,...,0.5,0.5,184.0,82,-1500.0,unassigned,LAX,4031.0,1.641023e+09,119420_01/01/2022


# Adding time of day column to IFF

In [20]:
iff_df['valid'] = pd.to_datetime(iff_df['recTime'], unit='s').dt.floor('H')
iff_df['datetime'] = pd.to_datetime(iff_df['recTime'], unit='s')

In [21]:
iff_df.sort_values('valid', ascending = True).head(1)

,fltKey,AcId,acType,Orig,Dest,perfCat,pKey_fltAcMode,coord1,coord2,alt,...,groundSpeed,course,rateOfClimb,scratchPad,airportCode,trackNumber,recTime,mergeKey,valid,datetime
0,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93549,-118.5447,21.69,...,184.0,82,-1500.0,unassigned,LAX,4031.0,1.641023e+09,119420_01/01/2022,2022-01-01 07:00:00,2022-01-01 07:47:59


In [22]:
metar_df.sort_values('valid').head(1)

,index,station,valid,lon,lat,elevation,tmpf,dwpf,relh,drct,...,wxcodes,ice_accretion_1hr,ice_accretion_3hr,ice_accretion_6hr,peak_wind_gust,peak_wind_drct,peak_wind_time,feel,metar,snowdepth
0,0,LAX,2022-01-01,-118.3865,33.9382,32.0,57.0,48.90,74.27,280.00,...,M,M,M,M,M,M,M,57.00,KLAX 010053Z 28012KT 10SM FEW015 SCT038 14/09 ...,M


# Removing null columns

In [23]:
null_cols = metar_df.columns[metar_df.isnull().all()]
metar_df = metar_df.drop(null_cols, axis=1)

Trimming down to usable columns

In [24]:
metar_trimmed = metar_df[['valid', 'lon', 'lat', 'elevation', 'tmpf', 'dwpf', 'relh', 'drct',
                         'sknt', 'alti', 'mslp', 'vsby', 'gust']]

Removing duplicate rows

In [25]:
metar_trimmed = metar_trimmed.drop_duplicates(subset='valid', keep='last')

# Adding METAR columns to IFF

In [26]:
iff_df.shape, metar_trimmed.shape

((1618440, 23), (8735, 13))

In [27]:
iff_df = pd.merge(iff_df, metar_trimmed, on='valid', how='left')

In [28]:
iff_df.shape

(1618440, 35)

In [29]:
iff_df['tmpf'].isna().sum()

0

# Attempting to find 4 conditions that meet criteria for a descent

* Condition 1: Change of MSL altitude over last 10 seconds is between -10 and -500
* Condition 2: Fraction of Maximum MSL altitude is > than .5
* Condition 3: Time until max deviation above 400 ft >= 0
* Condition 4: Descent Phase exists is true?

In [30]:
iff_df.head(1)

,fltKey,AcId,acType,Orig,Dest,perfCat,pKey_fltAcMode,coord1,coord2,alt,...,elevation,tmpf,dwpf,relh,drct,sknt,alti,mslp,vsby,gust
0,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93549,-118.5447,21.69,...,32.0,54.0,33.10,44.92,20.00,6.00,29.87,1011.40,10.0,M


* Converting altitude from 100s of ft to ft
* Converting rate of climb from ft/min to ft/sec

CHANGE BACK WHEN PLOTTING

In [31]:
iff_df['alt'] = iff_df['alt'].apply(lambda ft: ft * 100)
iff_df['rateOfClimb'] = iff_df['rateOfClimb'].apply(lambda time: time * 60)

In [32]:
iff_df['rateOfClimb'].describe()

count    1.618440e+06
mean    -4.694243e+03
std      3.519826e+04
min     -1.800000e+06
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.800000e+06
Name: rateOfClimb, dtype: float64

In [33]:
iff_df['alt'].describe()

count    1.618440e+06
mean     1.738024e+02
std      3.227084e+02
min      0.000000e+00
25%      1.060000e+02
50%      1.060000e+02
75%      1.060000e+02
max      1.000000e+04
Name: alt, dtype: float64

In [34]:
iff_df[iff_df['alt'] == 0]

,fltKey,AcId,acType,Orig,Dest,perfCat,pKey_fltAcMode,coord1,coord2,alt,...,elevation,tmpf,dwpf,relh,drct,sknt,alti,mslp,vsby,gust
881615,3485,FDY801,C208,NaN,LAX,T,3485_FDY801_ad35ed,33.93613,-118.39700,0.0,...,32.0,73.0,63.00,70.87,170.00,4.00,29.95,1014.00,10.0,M
1342931,48915,SKW3454,E75L,NaN,LAX,U,48915_SKW3454_a15c54,33.95047,-118.41742,0.0,...,32.0,66.0,41.00,39.99,250.00,7.00,30.06,1017.90,10.0,M


# Adding a key to merge by

# Merging IFF DF with EV RD DF to add landing runway

In [35]:
ev_rd_df = pd.read_csv('Sherlock_data/Merged_Filtered_EV_RD.csv')
ev_rd_df = ev_rd_df.drop('Unnamed: 0', axis=1)
ev_rd_df.columns

Index(['lKey', 'cKey', 'StartDate', 'StartTime', 'tMidnightSecs', 'tStartSecs',
       'tStopSecs', 'tStart', 'tStop', 'Duration', 'Msn', 'AcId', 'AcType',
       'tEv', 'EvType', 'ObjClass', 'OldName', 'NewName', 'Lat', 'Lon', 'aEv',
       'cEv', 'vEv', 'rEv', 'DTD', 'FlD', 'DDT', 'FlT', 'EvNumInfo',
       'EvCharInfo', 'pKey', 'mergeKey', 'RecordTime', 'time_Duration', 'PC',
       'WC', 'OT', 'Airline', 'CT', 'Orig', 'EstOrig', 'TOFRwy', 'Dest',
       'EstDest', 'LNDRwy', 'ATAThdArr', 'tON'],
      dtype='object')

# Joining the two dataframes together to get the landing runway

In [36]:
ev_rd_df = ev_rd_df.drop_duplicates(subset=['mergeKey'], keep='first')
print(ev_rd_df.shape)

(7455, 47)


In [37]:
# Drop the ev_rd records where there are no LNDRwy information
ev_rd_df['LNDRwy'].isnull().any()

True

In [38]:
ev_rd_df['LNDRwy'].value_counts(dropna=False)

25L    3749
24R    3119
06R     313
24L     142
06L      46
25R      44
NaN      20
07L      13
07R       9
Name: LNDRwy, dtype: int64

In [39]:
# Drop the ev_rd records where there are no LNDRwy information
ev_rd_df = ev_rd_df[ev_rd_df['LNDRwy'].notna()]
print(ev_rd_df.shape)

(7435, 47)


In [40]:
iff_df.shape, ev_rd_df.shape

((1618440, 35), (7435, 47))

In [41]:
ev_rd_df['mergeKey']

0        4151_08/06/2022
3        4176_08/06/2022
6        4177_08/06/2022
9        4178_08/06/2022
12       4188_08/06/2022
              ...       
22363    4117_08/06/2022
22366    4118_08/06/2022
22369    4121_08/06/2022
22372    4147_08/06/2022
22375    3099_08/06/2022
Name: mergeKey, Length: 7435, dtype: object

In [42]:
# Find the differences between mergeKey in ev_df and iff
len(list(set(iff_df['mergeKey'].values.tolist()) - set(ev_rd_df['mergeKey'].values.tolist())))

269

In [43]:
merge_df = pd.merge(iff_df, ev_rd_df, on = 'mergeKey', how = 'left')

In [44]:
merge_df

,fltKey,AcId_x,acType,Orig_x,Dest_x,perfCat,pKey_fltAcMode,coord1,coord2,alt,...,Airline,CT,Orig_y,EstOrig,TOFRwy,Dest_y,EstDest,LNDRwy,ATAThdArr,tON
0,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93549,-118.54470,2169.0,...,ASA,AC,NaN,NaN,NaN,LAX,LAX,06R,28208.0,28226.0
1,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93710,-118.52927,1881.0,...,ASA,AC,NaN,NaN,NaN,LAX,LAX,06R,28208.0,28226.0
2,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93859,-118.51472,1656.0,...,ASA,AC,NaN,NaN,NaN,LAX,LAX,06R,28208.0,28226.0
3,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.94002,-118.50101,1406.0,...,ASA,AC,NaN,NaN,NaN,LAX,LAX,06R,28208.0,28226.0
4,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.94139,-118.48753,1213.0,...,ASA,AC,NaN,NaN,NaN,LAX,LAX,06R,28208.0,28226.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618435,39730,SIA7406,B77L,NaN,LAX,J,39730_SIA7406_769102,33.93449,-118.39335,106.0,...,SIA,AC,NaN,NaN,NaN,LAX,LAX,25L,28324.0,28337.0
1618436,39730,SIA7406,B77L,NaN,LAX,J,39730_SIA7406_769102,33.93445,-118.39335,106.0,...,SIA,AC,NaN,NaN,NaN,LAX,LAX,25L,28324.0,28337.0
1618437,39730,SIA7406,B77L,NaN,LAX,J,39730_SIA7406_769102,33.93443,-118.39336,106.0,...,SIA,AC,NaN,NaN,NaN,LAX,LAX,25L,28324.0,28337.0
1618438,39730,SIA7406,B77L,NaN,LAX,J,39730_SIA7406_769102,33.93437,-118.39336,106.0,...,SIA,AC,NaN,NaN,NaN,LAX,LAX,25L,28324.0,28337.0


In [45]:
iff_df['LNDRwy'] = merge_df['LNDRwy']

In [46]:
len(iff_df)

1618440

In [47]:
query = list(iff_df[iff_df['LNDRwy'].isna()]['mergeKey'].values)
print("Percentage of null LNDRwy in data:", len(query)/len(iff_df))

Percentage of null LNDRwy in data: 0.029863943056276415


In [48]:
# Drop the records without LNDRwy information because later calculations rely on LNDRwy info
iff_df = iff_df[iff_df['LNDRwy'].notna()]
print(iff_df.shape)

(1570107, 36)


In [49]:
iff_df.head(1)

,fltKey,AcId,acType,Orig,Dest,perfCat,pKey_fltAcMode,coord1,coord2,alt,...,tmpf,dwpf,relh,drct,sknt,alti,mslp,vsby,gust,LNDRwy
0,119420,ASA846,B738,NaN,LAX,J,119420_ASA846_a784d5,33.93549,-118.5447,2169.0,...,54.0,33.10,44.92,20.00,6.00,29.87,1011.40,10.0,M,06R


Function to calculate delta_MSL

In [50]:
flights = iff_df.groupby('mergeKey')
print(len(flights))

7435


In [55]:
iff_df['rateOfClimb'].value_counts()

 0.0        1410767
-25200.0      27324
 25200.0      23614
-68400.0      18086
-46800.0      13674
             ...   
-45960.0          1
-14820.0          1
-42180.0          1
-35700.0          1
 9660.0           1
Name: rateOfClimb, Length: 794, dtype: int64

In [56]:
TODLand = []
TOD_dt = []
TOD_key = []
non_lands = []
for idx, flight in flights:
    # Round number to choose for descent beginning
    timeOfDescent = flight[flight['rateOfClimb'] <= -250]['datetime'].values
    tod_pkey = flight[flight['rateOfClimb'] <= -250]['mergeKey'].values
    # 128 is the elevation of LAX
    landed = flight[flight['alt'] <= 128]['datetime'].values
    landed_pkey = flight[flight['alt'] <= 128]['mergeKey'].values
    if len(timeOfDescent) > 0 and len(landed) > 0:
        TOD_key.append(tod_pkey[0])
        TOD_dt.append(timeOfDescent)
        TODLand.append((timeOfDescent[0], landed[0]))
    else:
        non_lands.append(tod_pkey)
        non_lands.append(landed_pkey)
    # ev_tod[idx] = TODs

In [57]:
len(TOD_key)

7401

In [58]:
len(TOD_dt[0])

16

In [54]:
# Extract the top of descent records
tod_df = pd.DataFrame()
for x in range(len(TOD_key)):
    tod_df = pd.concat([tod_df, iff_df.loc[(iff_df['mergeKey'] == TOD_key[x]) & (iff_df['datetime'].isin(TOD_dt[x]))]])
print(tod_df.shape)

KeyboardInterrupt: 

In [59]:
tod_df['mergeKey'].nunique

<bound method IndexOpsMixin.nunique of 0         119420_01/01/2022
1         119420_01/01/2022
2         119420_01/01/2022
3         119420_01/01/2022
4         119420_01/01/2022
                ...        
897051      3725_08/06/2022
897052      3725_08/06/2022
897053      3725_08/06/2022
897054      3725_08/06/2022
897055      3725_08/06/2022
Name: mergeKey, Length: 342642, dtype: object>

In [ ]:
length = len(tod_df)
chunk_size = length//10
df_list = [tod_df[i:i+chunk_size] for i in range(0, length, chunk_size)]
for idx, df in enumerate(df_list):
    df.to_csv(f'Sherlock_data/chunked_IFF/IFF_TOD_{idx}.csv')

In [ ]:
# Explore the data not classified as a top of descent event
pkeys = []
for arr in non_lands:
    if len(arr) > 0:
        pkeys.append(set(arr))

In [ ]:
pkeys, len(pkeys)

In [ ]:
for key_set in pkeys:
    key_list = list(key_set) # convert set to list
    key_str = key_list[0] # extract string value from list
    print(key_str)
    print(iff_df[iff_df['pKey_fltAcMode'] == key_str].head(1)['datetime'])

In [60]:
len(TODLand)

7401

In [61]:
deltas = []
for desc, land in TODLand:
    dt_ns = land - desc
    dt_s = dt_ns/np.timedelta64(1, 's')
    # print(dt_s)
    deltas.append(dt_s)

In [62]:
np.mean(deltas)

169.84934468315092